In [ ]:
from AFEP_parse import *
from alchemlyb.preprocessing import subsampling
import os
from scipy.signal import correlate
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.optimize import curve_fit, leastsq
from IPython.display import display, Markdown

import warnings #Suppress future warnings from pandas.
warnings.simplefilter(action='ignore', category=FutureWarning)

## This notebook makes use of pyMBAR and Alchemlyb
See Shirts and Chodera (2008) for more details

"Statistically optimal analysis of samples from multiple equilibrium states" doi: 10.1063/1.2978177

# User parameters
## IMPORTANT: Make sure the temperature matches the temperature used to run the simulations.

In [ ]:
temperature = 303.15
decorrelate = True #Flag for automatic decorrelation
detectEQ = False #Flag for automatic equilibrium detection

# Process the Bound Data

In [ ]:
path='../AFEP-Bound-Decoupling/output/'
filename='*.fepout' #This can be a regex pattern if you have more than one fepout file

RT = 0.00198720650096 * temperature # RT in kcal/mol
fepoutFiles = glob(path+filename)
fepoutFiles = natsorted(fepoutFiles)
#print(f"Will process {len(fepoutFiles)} fepout files")

In [ ]:
u_nk, _ = readAndProcess(fepoutFiles, temperature, decorrelate, detectEQ) #u_nk stores the fep data
perWindow, cumulative = doEstimation(u_nk)

In [ ]:
forward, forward_error, backward, backward_error = doConvergence(u_nk) #Used later in the convergence plot

In [ ]:
changeAndError = f'\u0394G = {np.round(cumulative.BAR.f.iloc[-1]*RT, 1)}\u00B1{np.round(cumulative.BAR.errors.iloc[-1], 3)} kcal/mol'
Markdown('<font size=7>{}</font><br/>'.format(changeAndError))

In [ ]:
width = 8
height = 4
fig, axes = plotGeneral(cumulative, perWindow, pdfX, pdfY, width=width, height=height, PDFtype='KDE')
plt.savefig(f'{path}_bound_generalFigures.pdf')
#plt.savefig(f'{path}dG_{affix}.png', dpi=600)
plt.show()

fig, convAx = plt.subplots(1,1)
convAx = convergencePlot(convAx, forward*RT, forward_error*RT, backward*RT, backward_error*RT)
convAx.set(xlabel='Fraction of Simulation Time', ylabel=r'Total $\rm\Delta G_{\lambda}$ (kcal/mol)')
fig.set_figwidth(width*3/4)
fig.set_figheight(height)
convAx.legend()
plt.savefig(f'{path}_bound_convergence.pdf')
plt.show()